In [1]:
import main

ModuleNotFoundError: No module named 'train'

In [ ]:
experiment=""
prunint_rate=
epochs=100
...

In [ ]:
python *.py --epochs {epochs}

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.utils.prune as prune


def extract_mask(model_dict):

    new_dict = {}
    for key in model_dict.keys():
        if 'mask' in key:
            new_dict[key] = copy.deepcopy(model_dict[key])

    return new_dict


def pruning_model_local(model, px):
    print('Apply Unstructured L1 Pruning Globally (all conv layers)')
    parameters_to_prune =[]
    for name,m in model.named_modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            prune.l1_unstructured(m, 'weight', amount=px)

def pruning_model_global(model, px):

    print('Apply Unstructured L1 Pruning Globally (all conv layers)')
    parameters_to_prune =[]
    for name,m in model.named_modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            parameters_to_prune.append((m,'weight'))
    parameters_to_prune = tuple(parameters_to_prune)
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=px,
    )
    
    
# network
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc = nn.Sequential(
            nn.Linear(10, 12),  nn.BatchNorm1d(12),  nn.ReLU(),   nn.Dropout(p=0.1), 
            nn.Linear(12, 12),  nn.BatchNorm1d(12),  nn.ReLU(),   nn.Dropout(p=0.1), 
            nn.Linear(12, 10), nn.Sigmoid()
        )
    
    def forward(self, input):
        x = self.fc(input)
        return x

In [7]:
net = Net()
net.state_dict()
pruning_model_local(net,0.9)
net.state_dict()

Apply Unstructured L1 Pruning Globally (all conv layers)


OrderedDict([('fc.0.bias',
              tensor([-0.2514, -0.1456, -0.1319,  0.3126,  0.0876,  0.1025, -0.2780,  0.1867,
                       0.1283,  0.0476, -0.2565,  0.1362])),
             ('fc.0.weight_orig',
              tensor([[-0.0850, -0.0605,  0.0241,  0.2455, -0.1685,  0.0042, -0.1103,  0.3053,
                        0.0231,  0.2004],
                      [ 0.0072,  0.1974, -0.0352, -0.2868,  0.0284, -0.2498,  0.2829,  0.2936,
                       -0.2258,  0.2932],
                      [ 0.0466, -0.2413,  0.1982,  0.2682,  0.1070,  0.1908,  0.0689, -0.0923,
                       -0.3136, -0.0741],
                      [ 0.2600, -0.1268, -0.0695,  0.3001,  0.0640, -0.2135, -0.0772,  0.2192,
                       -0.0970, -0.2372],
                      [-0.2804, -0.2681,  0.1753, -0.2451, -0.3128,  0.0283,  0.0164, -0.1099,
                       -0.0400,  0.2710],
                      [-0.2332, -0.1388, -0.0018,  0.2058,  0.1676,  0.2677, -0.2914,  0.2871,
    